# Basics
Let us first take a look at what is inside the `ib_insync` package

In [1]:
import ib_insync
import numpy as np

np.array(ib_insync.__all__)

array(['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution',
       'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList',
       'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider',
       'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList',
       'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle',
       'AccountValue', 'TickData', 'TickByTickAllLast',
       'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick',
       'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib',
       'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel',
       'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem',
       'Position', 'Fill', 'OptionComputation', 'OptionChain',
       'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick',
       'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option',
       'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity',
       'Bond', 'FuturesOption', 'MutualFund

## Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second starts an event loop to keep the live notebook updated:

In [2]:
from ib_insync import *
util.startLoop()

*Note that startLoop() only works in notebooks, not in regular Python programs.*

## Connecting
The main player of the whole package is the "IB" class. Let us create an IB instance and connect it to a running TWS/IBG application

In [3]:
ib = IB()
ib.connect('127.0.0.1', 1301, clientId=10)

<IB connected to 127.0.0.1:1301 clientId=10>

If the connection failed, then verify that the application has the API port enabled and double-check the hostname and port. For IB Gateway the default port is 4002. Make sure the clientID is not already in use.

If the connection succeeded, then ib will be `synchronized` with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). Let us list the current positions:

In [4]:
ib.positions()

[Position(account='DU275686', contract=Stock(conId=274105, symbol='SBUX', exchange='NASDAQ', currency='USD', localSymbol='SBUX', tradingClass='NMS'), position=100.0, avgCost=103.831603)]

Or filter the account values to get the liquidation value:

In [5]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU275686', tag='NetLiquidationByCurrency', value='1037737.9499', currency='BASE', modelCode='')]

The "current state" will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

## Contracts
Contracts can be specified in different ways:
* The ibapi way, by creating an `empty Contract` object and setting its attribute, one-by-one
* By using `Contract` and giving the attributes as keyword argument
* By using specialized `Stock`, `Option`, `Future`, `Forex`, `Index`, `CFD`, `Commodity`, `Bond`, `FuturesOption`, `MutualFund` or `Warrant` contracts


Some examples:

In [6]:
c = Contract()
c.conId = 270639
c

Contract(conId=270639)

In [7]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60')

Bond(secIdType='ISIN', secId='US03076KAA60')

## Sending a request
The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example `reqContractDetails`.

In [8]:
contract = Stock('TSLA', 'SMART', 'USD')
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='STK', conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALEODD,SCALERST,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,EDGX,FOXRIVER,PEARL,TPLUS1,NYSENAT,LTSE,MEMX,PSX', priceMagnifier=1, underConId=0, longName='TESLA INC', contractMonth='', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='US/Eastern', tradingHours='20201

## Current state vs request
Doing a request involves network traffic going up and down and can take considerable time. The current state on the other hand is always immediately available. So, it is preferable to use the current state mentods over requests. For example, use `ib.openOrder()` instead of `ib.reqOpenOrders()` or `ib.positions()` instead of `ib.reqPositions()`, etc.

In [9]:
%time l = ib.positions()

Wall time: 0 ns


In [10]:
%time l = ib.reqPositions()

Wall time: 2 ms


## Logging
The following will put log messages of INFO and higher level under the current active cell:

In [11]:
util.logToConsole()

To see all debug messages (including network traffic)

In [12]:
import logging
util.logToConsole(logging.INFO)

## Disconnecting
The following will disconnect `ib` and clear all its state:

In [13]:
ib.disconnect()

2020-12-28 22:12:59,891 ib_insync.ib INFO Disconnecting from 127.0.0.1:1301, 162 B sent in 9 messages, 16.6 kB received in 301 messages, session time 865 ms.
2020-12-28 22:12:59,892 ib_insync.client INFO Disconnecting
2020-12-28 22:12:59,896 ib_insync.client INFO Disconnected
